In [100]:
# Imports
import pandas as pd
from datetime import datetime
import itertools

tz_info = 'Europe/Madrid'

In [101]:
df = pd.read_excel('../data/Generalitat Extracción_2018.xlsx', skiprows=2)

In [102]:
df.head(10)

,CUPS,EG2,EG3,Fecha inicio Docu. cálculo,Fecha fin Docu. cálculo,Tarifa,Tipo Lectura,Indicadores,Consumo kWh ATR,Consumo kWh GLP,Consumo Kg
0,ES0217010002161818HX,BARCELONA,BADALONA,2017-12-16,2017-12-31,3.4 GAS,ESTIMADA,NaN,7237,0,0.0
1,ES0217010002161818HX,BARCELONA,BADALONA,2018-01-01,2018-01-12,3.4 GAS,REAL,NaN,63714,0,0.0
2,ES0217010002161818HX,BARCELONA,BADALONA,2018-01-13,2018-02-13,3.4 GAS,REAL,NaN,90071,0,0.0
3,ES0217010002161818HX,BARCELONA,BADALONA,2018-02-14,2018-03-13,3.4 GAS,REAL,NaN,74947,0,0.0
4,ES0217010002161818HX,BARCELONA,BADALONA,2018-03-14,2018-04-12,3.4 GAS,REAL,NaN,67338,0,0.0
5,ES0217010002161818HX,BARCELONA,BADALONA,2018-04-13,2018-05-14,3.4 GAS,REAL,NaN,42883,0,0.0
6,ES0217010002161818HX,BARCELONA,BADALONA,2018-05-15,2018-06-13,3.4 GAS,REAL,NaN,24175,0,0.0
7,ES0217010002161818HX,BARCELONA,BADALONA,2018-06-14,2018-07-13,3.4 GAS,REAL,NaN,18104,0,0.0
8,ES0217010002161818HX,BARCELONA,BADALONA,2018-07-14,2018-08-09,3.4 GAS,REAL,NaN,14070,0,0.0
9,ES0217010002161818HX,BARCELONA,BADALONA,2018-08-10,2018-09-17,3.4 GAS,REAL,NaN,21864,0,0.0


In [103]:
df['Fecha fin Docu. cálculo'] += pd.Timedelta(hours=23)

df['Fecha inicio Docu. cálculo'] = df['Fecha inicio Docu. cálculo'].dt.tz_localize(tz_info)
df['Fecha fin Docu. cálculo'] = df['Fecha fin Docu. cálculo'].dt.tz_localize(tz_info)

# datatime64 [ns] to unix time
df['measurementStart'] = df['Fecha inicio Docu. cálculo'].astype('int') / 10 ** 9
df['measurementStart'] = df['measurementStart'].astype('int')

df['measurementEnd'] = df['Fecha fin Docu. cálculo'].astype('int') / 10 ** 9
df['measurementEnd'] = df['measurementEnd'].astype('int')
df['ts'] = df['measurementStart']

# Calculate kWh
df['measurementValue'] = df['Consumo kWh ATR'].fillna(0) + df['Consumo kWh GLP'].fillna(0)

df = df[['CUPS', 'ts', 'measurementStart', 'measurementEnd', 'measurementValue', 'Tipo Lectura']]

In [104]:
def generate_date_range(row):
    init_date = datetime.fromtimestamp(row['measurementStart']).date()
    end_date = datetime.fromtimestamp(row['measurementEnd']).date()
    return pd.date_range(init_date, end_date).strftime("%Y-%m-%d").tolist()


In [128]:
for cups, sub_df in df.groupby('CUPS'):
    sub_df.sort_values(by=['measurementStart'], inplace=True)
    min_date = datetime.fromtimestamp(sub_df['measurementStart'].iloc[0]).date()
    max_date = datetime.fromtimestamp(sub_df['measurementEnd'].iloc[-1]).date()

    main_vect = (min_date, max_date)

    # aux_df = sub_df[['measurementStart', 'measurementEnd']].stack().diff(periods=1)

    sub_df['date_range'] = sub_df.apply(lambda x: generate_date_range(x), axis=1)

    total_range = pd.date_range(min_date, max_date).strftime("%Y-%m-%d").tolist()
    invoice_date_range = list(itertools.chain(*sub_df['date_range'].tolist()))

    for i in invoice_date_range:
        if i in total_range:
            total_range.remove(i)

    if len(total_range) > 0:
        print(total_range)